# LangChain PromptTemplate과 LCEL 체인 구성하기

이 노트북에서는 **PromptTemplate**을 사용하여 프롬프트를 구조화하고, **LCEL(LangChain Expression Language)**로 체인을 구성하는 방법을 알아봅니다.

## PromptTemplate이란?

프롬프트 템플릿은 동적으로 변경되는 값을 포함한 프롬프트를 정의하는 방법입니다.

```python
template = "안녕하세요, {name}님!"  # {name}이 변수
```

**장점:**
- 프롬프트 재사용성 향상
- 변수 부분만 교체하여 다양한 입력 처리
- 복잡한 프롬프트 구조화 가능

## LCEL(LangChain Expression Language)이란?

LCEL은 `|` (파이프) 연산자를 사용하여 컴포넌트를 연결하는 선언적 방식입니다.

```python
chain = prompt | model | output_parser
```

**장점:**
- 코드가 직관적이고 읽기 쉬움
- 스트리밍, 비동기 처리 자동 지원
- 컴포넌트 조합이 유연함

---

# 1. Ollama 설치 및 서버 실행

In [2]:
import subprocess
import time

# zstd 설치 (Ollama 설치의 사전 요구 사항)
!apt-get install -y zstd

# Ollama 설치
!curl -fsSL https://ollama.com/install.sh | sh

# 백그라운드에서 Ollama 서버 실행
subprocess.Popen(['ollama', 'serve'])

time.sleep(3)

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  zstd
0 upgraded, 1 newly installed, 0 to remove and 2 not upgraded.
Need to get 603 kB of archives.
After this operation, 1,695 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 zstd amd64 1.4.8+dfsg-3build1 [603 kB]
Fetched 603 kB in 0s (3,653 kB/s)
Selecting previously unselected package zstd.
(Reading database ... 117540 files and directories currently installed.)
Preparing to unpack .../zstd_1.4.8+dfsg-3build1_amd64.deb ...
Unpacking zstd (1.4.8+dfsg-3build1) ...
Setting up zstd (1.4.8+dfsg-3build1) ...
Processing triggers for man-db (2.10.2-1) ...
>>> Installing ollama to /usr/local
>>> Downloading ollama-linux-amd64.tar.zst
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current u

# 2. 모델 다운로드 & 패키지 설치

- `ollama pull llama3.2` - Llama 3.2 모델 다운로드
- `pip install langchain-ollama` - LangChain Ollama 통합 패키지 설치

# 3. PromptTemplate + LCEL 체인 실행

**코드 구조 설명:**

### 1) PromptTemplate 생성
```python
template = PromptTemplate.from_template('''...{context}...{question}...''')
```
- `{context}`, `{question}` - 나중에 값이 채워질 변수
- `from_template()` - 문자열에서 자동으로 변수 추출

### 2) LCEL 체인 구성
```python
chain = template | model
```
- `template` → 변수를 채워서 프롬프트 생성
- `model` → 생성된 프롬프트로 LLM 호출
- `|` 연산자가 데이터 흐름을 연결

### 3) 체인 실행
```python
chain.invoke({'context': '...', 'question': '...'})
```
- 딕셔너리로 변수 값 전달
- 체인이 순차적으로 실행되어 최종 응답 반환

In [5]:
!ollama pull llama3.2
!pip install -q langchain-ollama

---

## 코드 요약

| 단계 | 코드 | 설명 |
|------|------|------|
| 1 | `PromptTemplate.from_template()` | 변수가 포함된 프롬프트 템플릿 생성 |
| 2 | `ChatOllama(model='llama3.2')` | Ollama 채팅 모델 초기화 |
| 3 | `template \| model` | LCEL로 체인 구성 (프롬프트 → LLM) |
| 4 | `chain.invoke({...})` | 변수 값을 전달하여 체인 실행 |
| 5 | `response.content` | AIMessage에서 텍스트 추출 |

## 활용 팁

- **RAG 패턴**: Context에 검색된 문서를 넣으면 RAG(Retrieval-Augmented Generation) 구현 가능
- **출력 파서 추가**: `chain = template | model | StrOutputParser()`로 문자열만 추출
- **다른 템플릿 클래스**: `ChatPromptTemplate`은 역할별 메시지 구성에 적합

ChatOllama 모델 추가.        
template | model로 LCEL 체인 구성.   
response.content로 실제 답변 텍스트 출력.   